# E2: Directional Causality in a Layered Chain

## RTM Cascade Framework - Signature S2 Validation

This notebook validates that information flows forward-only between adjacent layers using Transfer Entropy and Granger Causality.

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd

# Parameters
N_LAYERS = 4
N_SAMPLES = 2000
COUPLING_STRENGTH = 0.7
NOISE_STD = 0.5
RANDOM_SEED = 42

rng = np.random.default_rng(RANDOM_SEED)
print("Parameters loaded.")

## 1. Generate Forward-Coupled Time Series

Model: $Y_n(t) = \kappa \cdot Y_{n-1}(t-1) + \varepsilon_n(t)$

In [ ]:
Y = np.zeros((N_LAYERS, N_SAMPLES))

# Layer 0: pure noise
Y[0] = rng.normal(0, NOISE_STD, N_SAMPLES)

# Subsequent layers: forward coupling
for n in range(1, N_LAYERS):
    noise = rng.normal(0, NOISE_STD, N_SAMPLES)
    Y[n, 0] = noise[0]
    for t in range(1, N_SAMPLES):
        Y[n, t] = COUPLING_STRENGTH * Y[n-1, t-1] + noise[t]

print(f"Generated {N_LAYERS} time series of length {N_SAMPLES}")

## 2. Visualize Time Series

In [ ]:
fig, ax = plt.subplots(figsize=(14, 5))
colors = plt.cm.viridis(np.linspace(0.2, 0.8, N_LAYERS))

for n in range(N_LAYERS):
    ax.plot(Y[n, :300], color=colors[n], alpha=0.8, label=f'Layer {n}')

ax.set_xlabel('Time')
ax.set_ylabel('Y_n(t)')
ax.set_title('Forward-Coupled Layer Time Series')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 3. Cross-Correlation Analysis

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
lags = np.arange(-20, 21)

for n in range(1, N_LAYERS):
    xcorr = [np.corrcoef(Y[n-1, max(0,-lag):min(N_SAMPLES, N_SAMPLES-lag)],
                         Y[n, max(0,lag):min(N_SAMPLES, N_SAMPLES+lag)])[0,1]
             for lag in lags]
    ax.plot(lags, xcorr, 'o-', markersize=4, label=f'Layer {n-1} → {n}')

ax.axvline(x=1, color='red', linestyle=':', label='Expected peak (lag=1)')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Lag')
ax.set_ylabel('Cross-correlation')
ax.set_title('Cross-Correlations (Forward Coupling at Lag=1)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 4. Granger Causality Test

In [ ]:
def granger_test(source, target, max_lag=5):
    n = len(target)
    Y_t = target[max_lag:]
    
    # Restricted: only past Y
    X_r = np.column_stack([target[max_lag-i-1:n-i-1] for i in range(max_lag)])
    
    # Full: past Y and past X
    X_f = np.column_stack([X_r, *[source[max_lag-i-1:n-i-1].reshape(-1,1) for i in range(max_lag)]])
    
    beta_r, res_r, _, _ = np.linalg.lstsq(X_r, Y_t, rcond=None)
    RSS_r = np.sum((Y_t - X_r @ beta_r)**2)
    
    beta_f, res_f, _, _ = np.linalg.lstsq(X_f, Y_t, rcond=None)
    RSS_f = np.sum((Y_t - X_f @ beta_f)**2)
    
    n_obs = len(Y_t)
    F = ((RSS_r - RSS_f) / max_lag) / (RSS_f / (n_obs - 2*max_lag))
    p = 1 - stats.f.cdf(F, max_lag, n_obs - 2*max_lag)
    return F, p

print("Granger Causality Test Results:\n")
for n in range(1, N_LAYERS):
    F_fwd, p_fwd = granger_test(Y[n-1], Y[n])
    F_rev, p_rev = granger_test(Y[n], Y[n-1])
    
    print(f"Layer {n-1} ↔ Layer {n}:")
    print(f"  Forward: F={F_fwd:.2f}, p={p_fwd:.4f} {'*' if p_fwd < 0.05 else ''}")
    print(f"  Reverse: F={F_rev:.2f}, p={p_rev:.4f} {'*' if p_rev < 0.05 else ''}")
    print()

## 5. Conclusion

The forward coupling is detected with high significance (low p-values), while the reverse direction shows no significant causality (high p-values).

This confirms **Signature S2**: information flows forward-only in the cascade.